
# Space

In [ ]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')

import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from config_observer.CKPD import ckpd_to_CkpdObsConfig


cohort_config = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_config['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_config['ObsDTName'] = 'ObsDT'
cohort_config['PID_ObsDT_columns'] = [cohort_config['RootID'], cohort_config['ObsDTName']]
print(cohort_config)

# 1. DsCase

In [ ]:
# 0. ************ RFT config ************
RecName_to_dsRec, RecName_to_dsRecInfo = {}, {}
cohort_label_list = [1]

# 1. ************ Case Trigger config ************
TriggerCaseMethod = 'TrulicityRx'


# 2. ************ InputCaseSetName ************
# option 1
# InputCaseSetName = 'C1.2.3-CGM5MinEntry'
# InputCaseSetName = 'sftcgmbf24haf2h-rs42-ds0.1-out0.1tstail0.1vd0.1'
InputCaseSetName = None 

In [ ]:
from recfldtkn.loadtools import fetch_trigger_tools

Trigger_Tools = fetch_trigger_tools(TriggerCaseMethod, SPACE)
case_id_columns = Trigger_Tools['case_id_columns']
cohort_config['case_id_columns'] = case_id_columns
case_id_columns

In [ ]:
from recfldtkn.pipeline_case import get_ds_case_to_process

InputCaseSetName, df_case = get_ds_case_to_process(InputCaseSetName, 
                                                   cohort_label_list, 
                                                   TriggerCaseMethod, 
                                                   cohort_config, 
                                                   SPACE, 
                                                   RecName_to_dsRec, 
                                                   RecName_to_dsRecInfo)

logger.info(f'InputCaseSetName: {InputCaseSetName}')
logger.info(f'df_case shape: {df_case.shape}')

In [ ]:
df_case = df_case.sample(1000)
df_case.shape

# 2. Tagging

In [ ]:
from recfldtkn.pipeline_case import process_df_tagging_tasks_in_chunks
from config_observer.QCF import cf_to_QueryCaseFeatConfig


CASE_TAGGING_PROC_CONFIG = {
    'use_CF_from_disk': False,
    'use_CO_from_disk': False,
    'start_chunk_id': 0,
    'end_chunk_id': None,
    'chunk_size': 500000,
    'save_to_pickle': False,
    'num_processors': 1
}

######################################
TagMethod_List = ['PttBasicDF', 'EgmBf1Y']
######################################

if len(TagMethod_List) > 0:
    logger.info(f'df_case shape: {df_case.shape}')
    OutputCaseSetName, df_case = process_df_tagging_tasks_in_chunks(df_case, cohort_label_list, case_id_columns, 
                                                                    InputCaseSetName, 
                                                                    TagMethod_List, cf_to_QueryCaseFeatConfig, 
                                                                    cohort_config, SPACE, 
                                                                    RecName_to_dsRec, RecName_to_dsRecInfo,
                                                                    **CASE_TAGGING_PROC_CONFIG)
    logger.info(f'df_case shape: {df_case.shape}')


else:
    pass



In [ ]:
df_case

# Step 1: fPttBasicDF

In [ ]:
df_case.columns

In [ ]:
df_case['patient_gender'].value_counts()

In [ ]:
df_case['patient_age_bucket'].value_counts()

In [ ]:
##################################
FilterMethod = 'fPttBasicDF'
##################################
import inspect

def fn_case_filtering(df_case):
    df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
# # Trigger2LearningCaseMethod
# pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
# module = load_module_variables(pypath)
# fn_case_filtering = module.fn_case_filtering

In [ ]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

## Step 2: fTailObsDT

In [ ]:
##################################
FilterMethod = 'fTailObsDT'
##################################
import inspect

def fn_case_filtering(df_case):
    series_time = df_case['ObsDT']
    stop_date = series_time.max() - pd.to_timedelta(10, unit='d')
    df_case = df_case[df_case['ObsDT'] <= stop_date].reset_index(drop = True)
    # df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

In [ ]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)